### Connect go Google Drive

In [ ]:
# mount google drive to access cloned repo
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/GitHub/finetuning-llm/

/content/drive/MyDrive/GitHub/finetuning-llm


In [ ]:
pwd

'/content/drive/MyDrive/GitHub/finetuning-llm'

### Install Dependencies

In [ ]:
!pip install datasets
!pip install trl
!pip install bitsandbytes
!pip install evaluate
!pip install rouge_score

# Optimised Code

In [ ]:
# Provide Colab a systemic path:
import sys
sys.path.append('/content/drive/My Drive/GitHub/finetuning-llm/')

In [ ]:
import torch
import os

from src.train.callbacks import BatchSizeCallback, MetricsLoggingCallback
from src.train.finetune_helpers import ModelArguments, ScriptArguments
from src.train.finetune_causallm import *

os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # log all model checkpoints

In [ ]:
model_args = ModelArguments(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    use_4bit=True,
    use_nested_quant=True,
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_quant_dtype="nf4",
    lora_alpha=128,
    lora_dropout=0.1,
    lora_r=8
)

script_args = ScriptArguments(
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    # auto_find_batch_size=True,
    # gradient_accumulation_steps=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    max_seq_length=512,
    dataset_name="4DR1455/finance_questions",
    bf16=True,
    optim='adafactor',
    lr_scheduler_type='cosine',
    packing=False,
    num_train_epochs=3,
    # max_steps=5,
    save_steps=100,
    logging_steps=10,
    eval_steps=100,
    warmup_steps=100,
    eval_strategy='steps',
    run_name="TinyLlama-1.1B-Chat-Finance-v1.3",
    report_to="wandb",
    save_safetensors=True,
    label_names=['labels'],
    load_best_model_at_end=True,
    dataloader_num_workers=10,
)


In [ ]:
causallm_llm = FinetuneCausalLM(model_args=model_args, script_args=script_args)

tokenizer = causallm_llm.get_tokenizer()
peft_config = causallm_llm.get_peft_config()
peft_model = causallm_llm.get_model(peft_config)

data_collator = data_collator(tokenizer=tokenizer)

train_set, eval_set, test_set = causallm_llm.tokenize_split_dataset(sample=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now default to True since model is quantized.


trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


[05:10:45] None                                                                            finetune_causallm.py:188

[05:10:49] Train set size: 43149                                                           finetune_causallm.py:261

           Eval set size: 5394                                                             finetune_causallm.py:262

           Test set size: 5394                                                             finetune_causallm.py:263

In [ ]:
causallm_llm.train(
    peft_model=peft_model,
    peft_config=peft_config,
    data_collator=data_collator)

/content/drive/My Drive/GitHub/finetuning-llm/src/train/finetune_causallm.py:307: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Tokenizing train dataset:   0%|          | 0/43149 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2244 > 2048). Running this sequence through the model will result in indexing errors


Tokenizing train dataset:   0%|          | 0/43149 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/5394 [00:00<?, ? examples/s]

...

[12:22:02] Evaluation metrics: {'eval_loss': 1.7270931005477905, 'eval_R1': 0.4508, 'eval_R2':      callbacks.py:39
           0.2045, 'eval_RL': 0.3714, 'eval_RLsum': 0.3928, 'eval_runtime': 392.5277,                              
           'eval_samples_per_second': 13.742, 'eval_steps_per_second': 0.431}                                      

wandb: Adding directory to artifact (./results/TinyLlama/TinyLlama-1.1B-Chat-v1.0/checkpoint-900)... Done. 0.0s


[13:09:52] Evaluation metrics: {'eval_loss': 1.7270501852035522, 'eval_R1': 0.4508, 'eval_R2':      callbacks.py:39
           0.2045, 'eval_RL': 0.3716, 'eval_RLsum': 0.393, 'eval_runtime': 391.9503,                               
           'eval_samples_per_second': 13.762, 'eval_steps_per_second': 0.431}                                      

wandb: Adding directory to artifact (./results/TinyLlama/TinyLlama-1.1B-Chat-v1.0/checkpoint-1000)... Done. 0.1s
wandb: Adding directory to artifact (./results/TinyLlama/TinyLlama-1.1B-Chat-v1.0/checkpoint-1011)... Done. 0.0s


[13:14:27] Training completed.                                                             finetune_causallm.py:330

In [ ]:
torch.cuda.device_count()

1

In [ ]:
os.cpu_count()

12

In [ ]:
train_set_subset = train_set.select(range(2))
eval_set_subset = eval_set.select(range(2))
test_set_subset = train_set.select(range(2))

In [ ]:
causallm_llm.train_set = train_set_subset
causallm_llm.eval_set_subset = eval_set_subset
causallm_llm.test_set_subset = test_set_subset


In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    # Constructing a standard Alpaca (https://github.com/tatsu-lab/stanford_alpaca#data-release) prompt
    mssg = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    for i in range(len(example["instruction"])):
        text = f"{mssg}\n### Instruction:\n{example['instruction'][i]}\n### Output: {example['output'][i]}"
        output_texts.append(text)
    return output_texts